<a href="https://colab.research.google.com/github/jimbrig-work/Work-Vault/blob/main/Getting_started_with_BigQuery_Modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Provide your credentials to the runtime

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=MVXbrd74I4JaWFImSXutAFL6uWI8Qe&prompt=consent&access_type=offline&code_challenge=2rUBFHI-HaMuoBljca5ivLBUNRivi8oc0v6wqfPltk8&code_challenge_method=S256

Enter verification code: 4/1AX4XfWj9NxGwhsBb-A09bZn8leAMkXxgwTjTfwn1lJjwmpIx5PmhTsytK1Y


## Optional: Enable data table display

Colab includes the ``google.colab.data_table`` package that can be used to display large pandas dataframes as an interactive data table.
It can be enabled with:

In [ ]:
%load_ext google.colab.data_table

If you would prefer to return to the classic Pandas dataframe display, you can disable this by running:
```python
%unload_ext google.colab.data_table
```

# Use BigQuery via magics

The `google.cloud.bigquery` library also includes a magic command which runs a query and either displays the result or saves it to a variable as a `DataFrame`.

In [4]:
# Display query output immediately

%%bigquery --project pg-us-e-app-584489
SELECT 
  COUNT(*) as total_rows
FROM `Equitable.ProRata`

,total_rows
0,1048575


In [ ]:
# Save output in a variable `df`
%%bigquery --project pg-us-e-app-584489 df
SELECT * FROM `Equitable.ProRata`

In [ ]:
df.head(10)

,Year_curr,Month_curr,curr_ss_date,Q,Life_Benefit,With_Map,ITMband,product,product_series,legacy_nonleg,Buyout_Indicator,db_benefit,tax_status,channel,pt_nlg_ind,Withdrawal_Taken,ProRata_ind,Rate_ib_db,AAge_band,Venerable_Ind,_49_1_Indicator,Inforce,Inforce_AV,Inforce_BB,Expected_Pro_Rata_Amount,Actual_Pro_Rata_Amount,Expected
0,2016,1,2016-01-29,2016Q1,DB_Only,Other,-50% and Less,ACC,ACC_1995/1996,Legacy,No_Buyout,RAT/ROP,NQ,R,False,0,0,6.0,67,EQH,EQH,1,74559.96,0.0,99.518907,0.0,EI
1,2016,1,2016-01-29,2016Q1,DB_Only,Other,-50% and Less,ACC,ACC_1995/1996,Legacy,No_Buyout,RAT/ROP,NQ,R,False,0,0,6.0,69,EQH,EQH,1,196082.82,0.0,278.176161,0.0,EI
2,2016,1,2016-01-29,2016Q1,DB_Only,Other,-50% and Less,ACC,ACC_1995/1996,Legacy,No_Buyout,RAT/ROP,NQ,R,False,0,0,6.0,75,EQH,EQH,1,41972.32,0.0,70.104267,0.0,EI
3,2016,1,2016-01-29,2016Q1,DB_Only,Other,-50% and Less,ACC,ACC_1995/1996,Legacy,No_Buyout,RAT/ROP,NQ,R,False,0,0,6.0,76,EQH,EQH,1,22135.57,0.0,37.899785,0.0,EI
4,2016,1,2016-01-29,2016Q1,DB_Only,Other,-50% and Less,ACC,ACC_1995/1996,Legacy,No_Buyout,RAT/ROP,NQ,R,False,0,0,6.0,77,EQH,EQH,3,506724.17,0.0,888.836421,0.0,EI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2016,2,2016-02-29,2016Q1,DB_Only,Other,40% to 50%,ACC,ACC_Oct. 1996 - pre '02,Legacy,No_Buyout,RAT/ROP,Q,W,False,1,1,0.0,80,EQH,EQH,2,98782.77,0.0,491.551295,1200.0,GAAP
1048571,2016,2,2016-02-29,2016Q1,DB_Only,Other,50% to 60%,ACC,ACC_Oct. 1996 - pre '02,Legacy,No_Buyout,RAT/ROP,NQ,W,False,1,1,0.0,86+,EQH,EQH,1,26420.81,0.0,64.744195,500.0,GAAP
1048572,2016,2,2016-02-29,2016Q1,DB_Only,Other,50% to 60%,ACC,ACC_Oct. 1996 - pre '02,Legacy,No_Buyout,RAT/ROP,NQ,W,False,1,1,0.0,86,EQH,EQH,1,33031.62,0.0,72.592490,202.0,GAAP
1048573,2016,2,2016-02-29,2016Q1,DB_Only,Other,60% to 75%,ACC,ACC_Oct. 1996 - pre '02,Legacy,No_Buyout,RAT/ROP,NQ,W,False,1,1,0.0,79,EQH,EQH,1,32003.47,0.0,62.385431,125.0,GAAP


In [ ]:
df.describe()

# Use BigQuery through google-cloud-bigquery

See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).

The [GSOD sample table](https://bigquery.cloud.google.com/table/bigquery-public-data:samples.gsod) contains weather information collected by NOAA, such as precipitation amounts and wind speeds from late 1929 to early 2010.


### Declare the Cloud project ID which will be used throughout this notebook

In [ ]:
project_id = 'pg-us-e-app-584489'

### Sample approximately 2000 random rows

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

sample_count = 2000
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `Equitable.ProRata`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `Equitable.ProRata`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)

### Describe the sampled data

In [ ]:
df.describe()

### View the first 10 rows

In [ ]:
df.head(10)

In [ ]:
df.sort_values('Year_curr', ascending=False).head(10)[['Life_Benefit', 'Expected_Pro_Rata_Amount', 'Actual_Pro_Rata_Amount']]

# Use BigQuery through pandas-gbq

The `pandas-gbq` library is a community led project by the pandas community. It covers basic functionality, such as writing a DataFrame to BigQuery and running a query, but as a third-party library it may not handle all BigQuery features or use cases.

[Pandas GBQ Documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_gbq.html)

In [ ]:
import pandas as pd

sample_count = 2000
df = pd.io.gbq.read_gbq('''
  SELECT name, SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''', project_id=project_id, dialect='standard')

df.head()

,name,count
0,James,272793
1,John,235139
2,Michael,225320
3,Robert,220399
4,David,219028


# Syntax highlighting
`google.colab.syntax` can be used to add syntax highlighting to any Python string literals which are used in a query later.

In [ ]:
from google.colab import syntax
query = syntax.sql('''
SELECT
  COUNT(*) as total_rows
FROM
  `bigquery-public-data.samples.gsod`
''')

pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')

,total_rows
0,114420316
